# Fine-tuning a Model for Masked Language Modeling (MLM) Exam

In this exam, you will be tasked with performing dataset preprocessing and fine-tuning a model for a masked language modeling task. Complete each step carefully according to the instructions provided.

### Model and Dataset Information

For this task, you will be working with the following:

- **Model Checkpoint**: Use the pre-trained model checkpoint `bert-base-uncased` for both the model and tokenizer.
- **Dataset**: You will be using the `CUTD/math_df` dataset. Ensure to load and preprocess the dataset correctly for training and evaluation.

**Note:**
- Any additional steps or methods you include that improve or enhance the results will be rewarded with bonus points if they are justified.
- The steps outlined here are suggestions. You are free to implement alternative methods or approaches to achieve the task, as long as you explain the reasoning and the process at the bottom of the notebook.
- You can use either TensorFlow or PyTorch for this task. If you prefer TensorFlow, feel free to use it when working with Hugging Face Transformers.
- The number of data samples you choose to work with is flexible. However, if you select a very low number of samples and the training time is too short, this could affect the evaluation of your work.

## Step 1: Load the Dataset

Load the dataset and split it into training and test sets. Use 20% of the data for testing.

In [18]:
!pip install transformers torch datasets

In [19]:
from datasets import load_dataset

ds = load_dataset("CUTD/math_df")

###Load train

In [20]:
ds = ds['train']

###Show data:

In [21]:
ds

Dataset({
    features: ['Unnamed: 0', 'text'],
    num_rows: 10000
})

###Rename coulmn Label

In [22]:
ds = ds.rename_column('Unnamed: 0', 'label')

####Show text

In [23]:
ds['text']

["A software engineer who disagrees with the established computer scientist's methodologies and approaches",
 'A German literature college student who is interested in historical contexts of works and the personalities of authors, but normally dislikes theatrical works.',
 'An entrepreneur who is developing a mobile app that utilizes satellite imagery to track population density for market research',
 'A pragmatic politician always considering the budget and community opinions before approving projects',
 "An experienced film director who recognizes the screenwriter's talent and frequently collaborates with them on projects",
 'A human rights lawyer who works closely with the journalist to gather evidence and advocate for prison reform',
 'A wilderness guide who frequently seeks physical therapy to recover from outdoor injuries',
 'A passionate curator at a civil war museum',
 'An old-fashioned fabric salesman who disagrees with the shift to digital marketing in the retail industry',
 

####Show label

In [24]:
ds['label']

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,


###Spilt the data

In [25]:
dataset = ds.train_test_split(test_size=0.2)
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['label', 'text'],
        num_rows: 8000
    })
    test: Dataset({
        features: ['label', 'text'],
        num_rows: 2000
    })
})


## Step 2: Load the Pretrained Model and Tokenizer

Use a pre-trained model and tokenizer for this task. Initialize both in this step.

In [26]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForMaskedLM

tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-uncased")
model = AutoModelForMaskedLM.from_pretrained("google-bert/bert-base-uncased")

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of the model checkpoint at google-bert/bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initial

## Step 3: Preprocess the Dataset

Define a preprocessing function that tokenizes the text data and prepares the inputs for the model. Ensure that you truncate the sequences to a maximum length of 512 tokens and pad them appropriately.

**Bonus**: If you performed more comprehensive preprocessing, such as removing links, converting text to lowercase, or applying additional preprocessing techniques.

In [27]:
import re
# Define a single function that combines text cleaning and tokenization
def preprocess_data(examples, tokenizer, max_length=512):

    # Define the cleaning function -> Bouns
    def clean_text(text):
        text = re.sub(r"http\S+|www\S+|https\S+", '', text) # Remove URLs
        text = re.sub(r'\s+', ' ', text) # Remove extra spaces
        text = text.lower() # Convert text to lowercase
        return text

    cleaned_texts = [clean_text(text) for text in examples['text']]

    inputs = tokenizer(cleaned_texts, truncation=True, padding='max_length', max_length=max_length, return_tensors='pt')

    inputs['labels'] = inputs['input_ids'].clone()

    return inputs

In [28]:
train_data = dataset['train'].map(preprocess_data, batched=True, remove_columns=["text"], fn_kwargs={'tokenizer': tokenizer})

Map:   0%|          | 0/8000 [00:00<?, ? examples/s]

In [29]:
train_data

Dataset({
    features: ['label', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
    num_rows: 8000
})

## Step 4: Define Training Arguments

Set up the training configuration, including parameters like learning rate, batch size, number of epochs, and weight decay.

In [30]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="no",
    learning_rate=3e-5,
    per_device_train_batch_size=8,
    num_train_epochs=1
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


## Step 5: Initialize the Trainer

Initialize the Trainer using the model, training arguments, and datasets (both training and evaluation).

In [31]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data
)

## Step 6: Fine-tune the Model

Run the training process using the initialized Trainer to fine-tune the model on the masked language modeling task.

In [32]:
trainer.train()

Step,Training Loss
500,0.346800
1000,0.000300


TrainOutput(global_step=1000, training_loss=0.1735583740770817, metrics={'train_runtime': 938.1111, 'train_samples_per_second': 8.528, 'train_steps_per_second': 1.066, 'total_flos': 2105638502400000.0, 'train_loss': 0.1735583740770817, 'epoch': 1.0})

## Step 7: Inference

Use the fine-tuned model for inference. Create a pipeline for masked language modeling and test it with a sample sentence.

In [33]:
from transformers import pipeline

fill_mask = pipeline(
    "fill-mask",
    model=model,
    tokenizer=tokenizer
)

sentence = "The biggest club in Spain is [MASK]."
result = fill_mask(sentence)

print(result)

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


[{'score': 0.33515995740890503, 'token': 7623, 'token_str': 'barcelona', 'sequence': 'the biggest club in spain is barcelona.'}, {'score': 0.03712981194257736, 'token': 27382, 'token_str': 'malaga', 'sequence': 'the biggest club in spain is malaga.'}, {'score': 0.029697587713599205, 'token': 0, 'token_str': '[PAD]', 'sequence': 'the biggest club in spain is.'}, {'score': 0.02921498566865921, 'token': 13083, 'token_str': 'valencia', 'sequence': 'the biggest club in spain is valencia.'}, {'score': 0.018681174144148827, 'token': 20901, 'token_str': 'airbus', 'sequence': 'the biggest club in spain is airbus.'}]
